# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [12]:
load_csv = "cities.csv"
city_data_df = pd.read_csv(load_csv)

city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Bredasdorp,-34.5322,20.0403,61.79,69,100,2.89,ZA,1669059916
1,1,Carnarvon,-24.8667,113.6333,73.62,57,39,8.93,AU,1669059916
2,2,Luganville,-15.5333,167.1667,81.25,81,69,5.73,VU,1669060002
3,3,Akdepe,42.0551,59.3788,37.49,68,100,4.38,TM,1669060002
4,4,Bethel,41.3712,-73.4140,40.01,39,0,12.66,US,1669060002


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [13]:
gmaps.configure(api_key=g_key)

city_data_df = city_data_df.dropna()
humidity = city_data_df["Humidity"].astype(float)

locations = city_data_df[["Lat", "Lng"]].astype(float)

humidity_fig = gmaps.figure(map_type='SATELLITE')

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max(cities_df["Humidity"]))

humidity_fig.add_layer(heat_layer)

humidity_fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [14]:
narrow_cities = city_data_df.copy()

narrow_cities = narrow_cities[narrow_cities['Max Temp'] > 21]
narrow_cities = narrow_cities[narrow_cities['Max Temp'] < 27]  
narrow_cities = narrow_cities[narrow_cities['Wind Speed'] < 4.5]
narrow_cities = narrow_cities[narrow_cities['Cloudiness'] == 0]

narrow_cities = narrow_cities.dropna()

narrow_cities = narrow_cities.reset_index()

del narrow_cities['index']

narrow_cities

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [15]:
hotel_df = []

for i in range(len(narrow_cities)):
    lat = narrow_cities.loc[i]['Lat']
    lng = narrow_cities.loc[i]['Lng']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    jsn = requested.json()
    try:
        hotel_df.append(jsn['results'][0]['name'])
    except:
        hotel_df.append("")
narrow_cities["Hotel Name"] = hotel_df
narrow_cities = narrow_cities.dropna(how='any')
narrow_cities.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name


In [16]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

AttributeError: 'list' object has no attribute 'iterrows'

In [17]:
# Add marker layer ontop of heat map
heatmap_fig = gmaps.figure()
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
heatmap_fig.add_layer(markers)
# Display figure
heatmap_fig

NameError: name 'hotel_info' is not defined